In [1]:
import time

debug = True

# display matrix picture with 0 and X
def matrix_overview(BB, bound):
    for ii in range(BB.dimensions()[0]):
        a = ('%02d ' % ii)
        for jj in range(BB.dimensions()[1]):
            a += '0' if BB[ii,jj] == 0 else 'X'
            a += ' '
        if BB[ii, ii] >= bound:
            a += '~'
        print a

def coppersmith_howgrave_univariate(pol, modulus, beta, mm, tt, XX):
    """
    Coppersmith revisited by Howgrave-Graham
    
    finds a solution if:
    * b|modulus, b >= modulus^beta , 0 < beta <= 1
    * |x| < XX
    """
    #
    # init
    #
    dd = pol.degree()
    nn = dd * mm + tt

    #
    # checks
    #
    if not 0 < beta <= 1:
        raise ValueError("beta should belongs in (0, 1]")

    if not pol.is_monic():
        raise ArithmeticError("Polynomial must be monic.")

    #
    # calculate bounds and display them
    #
    """
    * we want to find g(x) such that ||g(xX)|| <= b^m / sqrt(n)
    * we know LLL will give us a short vector v such that:
    ||v|| <= 2^((n - 1)/4) * det(L)^(1/n)
    * we will use that vector as a coefficient vector for our g(x)
    
    * so we want to satisfy:
    2^((n - 1)/4) * det(L)^(1/n) < N^(beta*m) / sqrt(n)
    
    so we can obtain ||v|| < N^(beta*m) / sqrt(n) <= b^m / sqrt(n)
    (it's important to use N because we might not know b)
    """
    if debug:
        # t optimized?
        print "\n# Optimized t?\n"
        print "we want X^(n-1) < N^(beta*m) so that each vector is helpful"
        cond1 = RR(XX^(nn-1))
        print "* X^(n-1) = ", cond1
        cond2 = pow(modulus, beta*mm)
        print "* N^(beta*m) = ", cond2
        print "* X^(n-1) < N^(beta*m) \n-> GOOD" if cond1 < cond2 else "* X^(n-1) >= N^(beta*m) \n-> NOT GOOD"
        
        # bound for X
        print "\n# X bound respected?\n"
        print "we want X <= N^(((2*beta*m)/(n-1)) - ((delta*m*(m+1))/(n*(n-1)))) / 2 = M"
        print "* X =", XX
        cond2 = RR(modulus^(((2*beta*mm)/(nn-1)) - ((dd*mm*(mm+1))/(nn*(nn-1)))) / 2)
        print "* M =", cond2
        print "* X <= M \n-> GOOD" if XX <= cond2 else "* X > M \n-> NOT GOOD"

        # solution possible?
        print "\n# Solutions possible?\n"
        detL = RR(modulus^(dd * mm * (mm + 1) / 2) * XX^(nn * (nn - 1) / 2))
        print "we can find a solution if 2^((n - 1)/4) * det(L)^(1/n) < N^(beta*m) / sqrt(n)"
        cond1 = RR(2^((nn - 1)/4) * detL^(1/nn))
        print "* 2^((n - 1)/4) * det(L)^(1/n) = ", cond1
        cond2 = RR(modulus^(beta*mm) / sqrt(nn))
        print "* N^(beta*m) / sqrt(n) = ", cond2
        print "* 2^((n - 1)/4) * det(L)^(1/n) < N^(beta*m) / sqrt(n) \n-> SOLUTION WILL BE FOUND" if cond1 < cond2 else "* 2^((n - 1)/4) * det(L)^(1/n) >= N^(beta*m) / sqroot(n) \n-> NO SOLUTIONS MIGHT BE FOUND (but we never know)"

        # warning about X
        print "\n# Note that no solutions will be found _for sure_ if you don't respect:\n* |root| < X \n* b >= modulus^beta\n"
    
    #
    # Coppersmith revisited algo for univariate
    #

    # change ring of pol and x
    polZ = pol.change_ring(ZZ)
    x = polZ.parent().gen()

    # compute polynomials
    gg = []
    for ii in range(mm):
        for jj in range(dd):
            gg.append((x * XX)**jj * modulus**(mm - ii) * polZ(x * XX)**ii)
    for ii in range(tt):
        gg.append((x * XX)**ii * polZ(x * XX)**mm)
    
    # construct lattice B
    BB = Matrix(ZZ, nn)

    for ii in range(nn):
        for jj in range(ii+1):
            BB[ii, jj] = gg[ii][jj]

    # display basis matrix
    if debug:
        matrix_overview(BB, modulus^mm)

    # LLL
    BB = BB.LLL()

    # transform shortest vector in polynomial    
    new_pol = 0
    for ii in range(nn):
        new_pol += x**ii * BB[0, ii] / XX**ii

    # factor polynomial
    potential_roots = new_pol.roots()
    print "potential roots:", potential_roots

    # test roots
    roots = []
    for root in potential_roots:
        if root[0].is_integer():
            result = polZ(ZZ(root[0]))
            if gcd(modulus, result) >= modulus^beta:
                roots.append(ZZ(root[0]))

    # 
    return roots

In [7]:
512-len('11010101110011101100010100100010')

480

In [10]:
# RSA gen
length_N = 1024;
p = next_prime(2^int(round(length_N/2)));
q = next_prime( round(pi.n()*p) );
N = p*q;


N = 91524376058435512802977349054137484853020749756475848937949032819271976245618954056085280356793284207373688914402063849900167769927943540024042052332072339128515647471594344561990787127600319799073223486596604605512823655072245578506874569093613146786306365882399107944523349237631503342093680744942454169241

# qbar is q + [hidden_size_random]
hidden = 490;
diff = ZZ.random_element(0, 2^hidden-1)
qbar = int("11010101110011101100010100100010????????1110010011101111000000100011011011010110011111010010111110110100110111010101101011111101010011011110011100001001101000100110111110001101101000000001011001111000001100010011000101110011101011100101001000010110011100001001010010110111100001010010110101010001010001110011111100101011011011001001100010101101010100000110011101100010101110000100001001000001110110000101100011001001????????00010010????????1010101110001001001101111111000001111011????????010111110111011000010111".replace('?','0'),2)

F.<x> = PolynomialRing(Zmod(N), implementation='NTL'); 
pol = x - qbar
dd = pol.degree()

# PLAY WITH THOSE:
beta = 0.4                             # we should have q >= N^beta
epsilon = beta / 7                     # <= beta/7
mm = ceil(beta**2 / (dd * epsilon))    # optimized
tt = floor(dd * mm * ((1/beta) - 1))   # optimized
XX = ceil(N**((beta**2/dd) - epsilon)) # we should have |diff| < X

# Coppersmith
start_time = time.time()
roots = coppersmith_howgrave_univariate(pol, N, beta, mm, tt, XX)

# output
print "\n# Solutions"
print "we found:", roots
print("in: %s seconds " % (time.time() - start_time))


# Optimized t?

we want X^(n-1) < N^(beta*m) so that each vector is helpful
* X^(n-1) =  1.13831562340388e190
* N^(beta*m) =  3.57967939805122e369
* X^(n-1) < N^(beta*m) 
-> GOOD

# X bound respected?

we want X <= N^(((2*beta*m)/(n-1)) - ((delta*m*(m+1))/(n*(n-1)))) / 2 = M
* X = 47428979768479754651633650237440
* M = 7.84466148277224e34
* X <= M 
-> GOOD

# Solutions possible?

we can find a solution if 2^((n - 1)/4) * det(L)^(1/n) < N^(beta*m) / sqrt(n)
* 2^((n - 1)/4) * det(L)^(1/n) =  2.79709917295080e359
* N^(beta*m) / sqrt(n) =  1.35299163722642e369
* 2^((n - 1)/4) * det(L)^(1/n) < N^(beta*m) / sqrt(n) 
-> SOLUTION WILL BE FOUND

# Note that no solutions will be found _for sure_ if you don't respect:
* |root| < X 
* b >= modulus^beta

00 X 0 0 0 0 0 0 ~
01 X X 0 0 0 0 0 
02 X X X 0 0 0 0 
03 X X X X 0 0 0 
04 0 X X X X 0 0 
05 0 0 X X X X 0 
06 0 0 0 X X X X 
potential roots: []

# Solutions
we found: []
in: 0.00933694839478 seconds 


In [11]:
# RSA Utilities
#
# Other factorization algorithms included with Sage:
# qsieve - The best algorithm for factoring numbers of the form \(pq\) up to
#          around 100 digits.
# ecm.factor - The best algorithm for factoring numbers of the form \(n=pm\),
#              where \(p\) is not “too big”.

def rsa_make_d(p, q, e):
    """Computes private exponent d given p, q and e.
    """
    return inverse_mod(e, (p - 1) * (q - 1))

def factor_rsa_modulus_n(N, e, d):
    """Factorize the RSA modulus N given the public and private exponents, e
    and d.

    Source: https://crypto.stanford.edu/~dabo/papers/RSA-survey.pdf
    CTF: BCTF 2016 crypto 500 Hyper RSA
    """
    k = d * e - 1
    while True:
        g = randint(2, N-1)
        t = k
        while t % 2 == 0:
            t = t // 2
            x = power_mod(g, t, N)
            y = gcd(x - 1, N)
            if x > 1 and y > 1:
                p = y
                q = N // y
                if p > q:
                    return (p, q)
                else:
                    return (q, p)

def factor_fermat(N):
    """Factorize N into p and q for p and q share half of their leading bits.
    i.e., if the gap between p and q is below the square root of p

    Source: http://facthacks.cr.yp.to/fermat.html
    CTF: BKP CTF 2016 Bob's Hat
    """
    if N <= 0: return [N]
    if is_even(N): return [2,N/2]
    a = ceil(sqrt(N))
    while not is_square(a^2 - N):
        a = a + 1
    b = sqrt(a^2-N)
    return [a - b,a + b]

def factor_lattice(N, nearp, howclose, t, k):
    """Finds p very quickly if p has about half as many bits as N and half of
    the leading bits of p are known.

    Source: http://facthacks.cr.yp.to/lattice.html
    """
    R.<x> = PolynomialRing(ZZ)
    f = howclose * x + nearp
    M = matrix(t)
    for i in range(t):
        M[i] = (f ^ i * N ^ max(k - i, 0)).coefficients(sparse=False) + [0] * (t - 1 - i)
    M = M.LLL()
    Q = sum(z * (x / howclose) ^ i for i, z in enumerate(M[0]))
    for r, multiplicty in Q.roots():
        if nearp + r > 0:
            g = gcd(N, nearp + r)
            if g > 1: return [g, N / g]
    return [1, N]

def factor_rsa_wiener(N, e):
    """Wiener's attack: Factorize the RSA modulus N given the public exponents
    e when d is small.

    Source: https://crypto.stanford.edu/~dabo/papers/RSA-survey.pdf
    CTF: BKP CTF 2016 Bob's Hat
    """
    N = Integer(N)
    e = Integer(e)
    cf = (e / N).continued_fraction().convergents()
    for f in cf:
        k = f.numer()
        d = f.denom()
        if k == 0:
            continue
        phi_N = ((e * d) - 1) / k
        b = -(N - phi_N + 1)
        dis = b ^ 2 - 4 * N
        if dis.sign() == 1:
            dis_sqrt = sqrt(dis)
            p = (-b + dis_sqrt) / 2
            q = (-b - dis_sqrt) / 2
            if p.is_integer() and q.is_integer() and (p * q) % N == 0:
                p = p % N
                q = q % N
                if p > q:
                    return (p, q)
                else:
                    return (q, p)

def modular_sqrt(a, p):
    """Returns modular square root of an integer number a modulo a prime number
    p.

    Source: http://www.mersennewiki.org/index.php/Modular_Square_Root
    """

    if a == 0:
        return (0, )

    if legendre_symbol(a, p) != 1:
        raise ValueError("a is not a quadratic residue modulo p")

    if p == 2:
        return (a % p,)

    if p % 4 == 3:
        r = power_mod(a, (p + 1) // 4, p)
        return (r, p - r)

    if p % 8 == 5:
        v = power_mod(2 * a, (p - 5) // 8, p)
        i = (2 * a * power_mod(v, 2, p)) % p
        r = a * v * (i - 1) % p
        return (r, p - r)

    if p % 8 == 1: # Shanks' method
        q = p - 1
        e = 0
        while q % 2 == 0:
            q //= 2
            e += 1
        while True:
            x = Integer(randint(2, p - 1))
            z = power_mod(x, q, p)
            if power_mod(z ^ 2, e - 1, p) != 1:
                break
        y = z
        r = e
        x = power_mod(a, (q - 1) // 2, p)
        v = (a * x) % p
        w = (v * x) % p
        while w != 1:
            k = 1
            ws = power_mod(w, 2, p)
            while power_mod(ws, k, p) != 1:
                k += 1
            d = power_mod(y ^ 2, r - k - 1, p)
            y = power_mod(d, 2, p)
            r = k
            v = (d * v) % p
            w = (w * y) % p
        return (v, p - v)

    raise ValueError("Cannot find modular square root")

def modular_sqrt_rabin(a, p, q):
    """Returns modular square root of an integer number a modulo the product of
    two primes p and q. (i.e. decryption of Rabin)

    CTF: HITCON 2015 Quals crypto 314 Rsabin
    """
    rp = modular_sqrt(a, p)
    rq = modular_sqrt(a, q)
    r = []
    for a in rp:
        for b in rq:
            r.append(crt(a, b, p, q))
    return r


In [12]:
def factor_lattice(N, nearp, howclose, t, k):
    """Finds p very quickly if p has about half as many bits as N and half of
    the leading bits of p are known.

    Source: http://facthacks.cr.yp.to/lattice.html
    """
    R.<x> = PolynomialRing(ZZ)
    f = howclose * x + nearp
    M = matrix(t)
    for i in range(t):
        M[i] = (f ^ i * N ^ max(k - i, 0)).coefficients(sparse=False) + [0] * (t - 1 - i)
    M = M.LLL()
    Q = sum(z * (x / howclose) ^ i for i, z in enumerate(M[0]))
    for r, multiplicty in Q.roots():
        if nearp + r > 0:
            g = gcd(N, nearp + r)
            if g > 1: return [g, N / g]
    return [1, N]

In [136]:
N = 83053629859775920711567812747186440767081481550602251188553309206324295681965330133627944485866485070999764911856916661666341055580506407138328928582102280441307404997028414990454411914585618511541665346003685339249951173834579019444763896855008180436531017231173187615328630282433643469979677583481033456767

In [137]:
precv = "11011101000001010000101110000011????????0000111000110101111001110101111110101000000101101100110100101100011111011100010011111101100100001010100010101010110010110011001111111110111110001111100000100110100100011000001111000001101100010111000111011110001101010001001010111000101010011011011101100011011001110011110011110101001100111010100111110000011101110001111110110011001111101011100000100001001111000010011111110000????????11001001????????0001111000100110010100000010011010100110????????010100111010010111111001"

In [138]:
pmsb = precv[:-96] + '0'*96

In [139]:
for i in range(0,256):

    ptry = pmsb.replace('????????','{:08d}'.format(int(bin(i)[2:])))
    #print(bin(i)[2:])

    facco = factor_lattice(N, int(ptry,2), 2**96, 8, 4)
    
    if facco[0] != 1:
        print(facco)
        p,q = facco
        
assert p*q == N

[11575741324989063940281303836117959708777860718967474194557642681569633215263644241397535181182103760190033820353082643135125681090364358617583219675211257, 7174800086495055188509357008521504972173899314795570630078666579402551551217580174534108146333750447573693730100331428261507221799217562896389605880165431]


In [140]:
phi = (p-1)*(q-1)
print(phi)
eulinv(0x10001,phi)


83053629859775920711567812747186440767081481550602251188553309206324295681965330133627944485866485070999764911856916661666341055580506407138328928582102261690765993512909286199793567275120937559781631582958860702940690201649812538220347965211680664582323253503622734201257233649530753888058163610655478080080


4688

In [58]:
# Get euler inverse of x (mod n)
def eulinv(x, n):
    a, b, u = 0, n, 1
    while x > 0:
        q = b // x 
        x, a, b, u = b % x, u, x, a - q * u
    if b == 1:
        return a % n
    else:
        return None

In [32]:
11311569296778257566316063650742214065753951899646801957249815519713146636234353174691717488107769399384511815687655336821304484753870319205933895268408483*12938916575228813185264301257311667599832892654521158841551461501955634358900410205906150664983958943206058704154691235425206496609371331530887497871974359

146359451465933527125559782999149121046288419152103871589339519138259935083096255623150327987836277883456939067381456086168237719721064739101950574394246224220015112632765697199795985974899913728296309176496996544130705773426055946528621157713576826203191211632295273915206873740865102625993036480670914087397

In [33]:
N

146359451465933527125559782999149121046288419152103871589339519138259935083096255623150327987836277883456939067381456086168237719721064739101950574394246224220015112632765697199795985974899913728296309176496996544130705773426055946528621157713576826203191211632295273915206873740865102625993036480670914087397

In [121]:
import hashlib
import numpy.random as npr

alp = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'

def poww(pow_hash):
    
    while True:
        
        pow_try = b'UMass-' + ''.join(npr.choice(list(alp),32)).encode()
        #print(pow_try)
        pow_sha = hashlib.sha256(pow_try).hexdigest().encode()
        
        if pow_hash == pow_sha[:5]:
            
            return pow_try
    
poww(b'bd7f3')

'UMass-NrBy7cqgoIAWopAxqWmgDF3kNTNCheEs'

In [ ]:
# Proof of work
	s.recvuntil(b'begins with ')


	pow_hash = s.recvuntil('\n', drop=True)
	s.recvuntil('> ')
	print(pow_hash)

	while True:

		pow_try = b'UMass-' + os.urandom(32)

		pow_sha = hashlib.sha256(pow_try).hexdigest().encode()

		if pow_hash == pow_sha[:5]:

			s.sendline(pow_try)
			break